In [34]:
import pandas as pd
import numpy as np
from grouprecommender import GroupRecommender

# Loading the Dataset

In [6]:
poorly_formatted_rows = [2120259, 2446317, 11141080, 11152098, 11152401, 11882086, 12902538, 12935043, 17589538]
df = pd.read_csv('lastfm_data/userid-timestamp-artid-artname-traid-traname.tsv', header=None, skiprows=poorly_formatted_rows, sep='\t')
df.columns = ['user_id', 'timestamp', 'artist_id', 'artist_name', 'track_id', 'track_name']
df.dropna(inplace=True)

# Group Recommendations

In [35]:
gr = GroupRecommender('utility_matrix.pickle')

## Obtaining Group Recommendations

In [87]:
user_ids = np.random.choice(df['user_id'].unique(), 3, replace=False)
max_recommendations = 1000
playlist = gr.full_recommendation(user_ids, max_recommendations, df)

In [116]:
print("Recommended Songs for the Group:", user_ids)
for i, track in enumerate(playlist):
    print(str(i) + ')' + track[0] + ' - ' + track[1])

Recommended Songs for the Group: ['user_000208' 'user_000084' 'user_000674']
0)Daft Punk - Technologic
1)Kelly Clarkson - Because Of You
2)Jamiroquai - Seven Days In Sunny June
3)Kylie Minogue - Can'T Get You Out Of My Head
4)Gorillaz - Feel Good Inc.
5)Red Hot Chili Peppers - Californication
6)Red Hot Chili Peppers - Otherside
7)Gorillaz - Dare
8)Daft Punk - One More Time
9)Keane - Somewhere Only We Know
10)Maroon 5 - This Love
11)Moby - Lift Me Up
12)Madonna - Hung Up
13)Afi - Miss Murder
14)Gorillaz - Clint Eastwood
15)Red Hot Chili Peppers - By The Way
16)Red Hot Chili Peppers - Charlie
17)The Killers - Somebody Told Me
18)Sean Paul - Temperature
19)Daft Punk - Digital Love
20)Madonna - Jump
21)Linkin Park - In The End
22)Daft Punk - Harder, Better, Faster, Stronger
23)Gorillaz - Feel Good Inc
24)Afi - Love Like Winter
25)Robbie Williams - Angels
26)Red Hot Chili Peppers - Dani California


## Evaluating Recommendations

In [3]:
#TODO: implement evaluation methods

## Creating Group Playlist

In [129]:
import spotipy
import pprint
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '5de79a00f1b9475098f277ce2a609120'
client_secret = '909633d1415345fe89882b6e398390df'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                          client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [115]:
spotify_track_ids = []
for track in playlist:
    search_str = track[0] + ' ' + track[1]
    result = sp.search(search_str, limit=1)
    spotify_track_ids.append(result['tracks']['items'][0]['id'])

In [ ]:
playlist_name = 'Group recommendations for '
playlist_name += ', '.join(user_ids)
username = 'franchenstein'
token = util.prompt_for_user_token(username, 
                                   client_id=client_id, 
                                   client_secret=client_secret, 
                                   redirect_uri='https://www.github.com/alexing10/musicmagal')
sp.Spotify(auth=token)
sp.user_playlist_create(username, playlist_name, '')

In [ ]:
sp.user_playlist_add_tracks(username, playlist_id, spotify_track_ids)